In [1]:
import pandas as pd
import numpy as np
import os
import sys
from collections import OrderedDict
import json

### valid data

In [2]:
valid_pub_info = pd.read_pickle('./pkl/valid_pub_info.pkl')
cna_valid_unass = pd.read_pickle('./pkl/cna_valid_unass.pkl')

In [3]:
valid_data = cna_valid_unass.merge(valid_pub_info, 'left', 'paper_id')

In [4]:
valid_data['author_idx'] = valid_data['author_idx'].astype(int)
print(valid_data['author_idx'].max())

3028


In [5]:
valid_data['author_name'] = valid_data.apply(lambda row: row['authors'][row['author_idx']]['name'], axis=1)

In [6]:
valid_data['author_org'] = valid_data.apply(lambda row: row['authors'][row['author_idx']].get('org'), axis=1)

In [7]:
valid_data = valid_data[['paper_id', 'author_name', 'author_org']]

In [8]:
def convert(name):
    name = name.lower()
    name = name.replace('. ', '_').replace('.', '_').replace(' ', '_').replace('-', '')
    if name in ['yang_jie', 'jie_yang_0002', 'jie\xa0yang', 'jie_yang_0008']:
        name = 'jie_yang'
    if name in ['liu_bing']:
        name = 'bing_liu'
    return name

valid_data['author_name'] = valid_data['author_name'].apply(convert)

In [9]:
valid_data.head()

,paper_id,author_name,author_org
0,F3Mha4HG,lei_shi,State Key Laboratory of Catalysis
1,ZSS7tPJq,jing_yu,Department of Materials Science and Engineerin...
2,NkHR5dsF,c_h_chen,Department of Physics and Astronomy
3,ihHKyfah,li_he,"School of Computer Science and Engineering, He..."
4,iEZVdy9V,min_chen,"Department of Materials Science, State Key Lab..."


In [10]:
valid_data['author_name'].nunique()

36

In [12]:
valid_data[valid_data['author_org'] == 'South China University of Technology']

,paper_id,author_name,author_org
5810,FyI9Y37Z,jie_yang,South China University of Technology
7036,trFiz7xt,rui_zhang,South China University of Technology
7431,BhUmfUOJ,qing_zhang,South China University of Technology


In [13]:
whole_author_name_paper_ids = pd.read_pickle('./pkl/whole_author_name_paper_ids.pkl')

In [14]:
whole_author_name_paper_ids.head()

,author_id,author_name,paper_ids
0,004mBKh6,changsheng_li,"[lRl7m2tU, 1vUnZLES, 5IbEOTRh, dVAKsHla, TY0MY..."
1,008b5hIc,yuanyuan_liu,[nPuf0kYs]
2,00BAgHFr,xi_li,"[TuEkjf5i, kn6wo3ot, 0f2awX7U]"
3,00M93JDI,weiping_cai,"[mHDfYkqE, RWISo90R, FSsnfI0X, 4U4GvMmc, Yh45T..."
4,00ShLPmg,hong_xiao,"[inFGp3mc, uyJus3zu, o6poFI1m, KvHObx7O, vg64a..."


In [15]:
valid_data_ = valid_data.merge(whole_author_name_paper_ids[['author_name', 'author_id']], 'left', 'author_name')

In [16]:
valid_data_.isnull().sum() / len(valid_data_)

paper_id       0.000000
author_name    0.000000
author_org     0.001193
author_id      0.000000
dtype: float64

In [18]:
valid_data_.head()

,paper_id,author_name,author_org,author_id
0,F3Mha4HG,lei_shi,State Key Laboratory of Catalysis,0gL4hj4n
1,F3Mha4HG,lei_shi,State Key Laboratory of Catalysis,0jknXeWe
2,F3Mha4HG,lei_shi,State Key Laboratory of Catalysis,1vE3Drg0
3,F3Mha4HG,lei_shi,State Key Laboratory of Catalysis,2IVNIGb6
4,F3Mha4HG,lei_shi,State Key Laboratory of Catalysis,5QgbLXhl


In [19]:
valid_data_.to_pickle('./pkl/valid_data.pkl')